In [51]:
%cd C:\Users\kabousselham\Desktop\python\Manning_sales_forecast_deep_learning

C:\Users\kabousselham\Desktop\python\Manning_sales_forecast_deep_learning


In [52]:
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
import pandas as pd
from pathlib import Path
from itertools import product
from scipy.signal import detrend
from tqdm.autonotebook import tqdm
np.random.seed()
tqdm.pandas()

In [53]:
data_path=Path("data")

# Answer to question 1

In [54]:
df=pd.read_csv(data_path/'sales.csv')

In [55]:
df.head()

,date,store_id,cat_id,sales
0,2011-01-29,TX_1,FOODS,3950.35
1,2011-01-30,TX_1,FOODS,3844.97
2,2011-01-31,TX_1,FOODS,2888.03
3,2011-02-01,TX_1,FOODS,3631.28
4,2011-02-02,TX_1,FOODS,3072.18


# Answers to question 2

In [56]:
#Showing the number of unique values of cat_id
df['cat_id'].nunique()

3

In [57]:
#Showing the number of unique values of store_id
df['store_id'].nunique()

10

In [58]:
# Showing the unique values of such pandas series one by one
df['cat_id'].unique()

array(['FOODS', 'HOBBIES', 'HOUSEHOLD'], dtype=object)

In [59]:
df['store_id'].unique()

array(['TX_1', 'TX_2', 'TX_3', 'CA_1', 'CA_2', 'CA_3', 'CA_4', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

# Answers to question 3

In [60]:
#check first the types of each column of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58230 entries, 0 to 58229
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      58230 non-null  object 
 1   store_id  58230 non-null  object 
 2   cat_id    58230 non-null  object 
 3   sales     58230 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.8+ MB


In [61]:
df["date"]=pd.to_datetime(df["date"], yearfirst=True)

In [62]:
min_date=df['date'].min()
max_date = df['date'].max()
print(f"Start and end dates of sales are:{min_date} and {max_date}")

Start and end dates of sales are:2011-01-29 00:00:00 and 2016-05-22 00:00:00


# Answers to question 4

In [63]:
# Check wether there is a duplicate date
stores=df['store_id'].unique()
categories=df['cat_id'].unique()
for i,j in product(stores,categories):
    print(f"Iteration for store {i} and category {j}:")
    control_df=df[(df['store_id']==i) & (df['cat_id']==j)]
    if control_df.duplicated(subset='date').any():
        print(f"Dataframe shows duplicate dates along stores {i} and category {j}")
    else:
        print("No duplicate dates")

Iteration for store TX_1 and category FOODS:
No duplicate dates
Iteration for store TX_1 and category HOBBIES:
No duplicate dates
Iteration for store TX_1 and category HOUSEHOLD:
No duplicate dates
Iteration for store TX_2 and category FOODS:
No duplicate dates
Iteration for store TX_2 and category HOBBIES:
No duplicate dates
Iteration for store TX_2 and category HOUSEHOLD:
No duplicate dates
Iteration for store TX_3 and category FOODS:
No duplicate dates
Iteration for store TX_3 and category HOBBIES:
No duplicate dates
Iteration for store TX_3 and category HOUSEHOLD:
No duplicate dates
Iteration for store CA_1 and category FOODS:
No duplicate dates
Iteration for store CA_1 and category HOBBIES:
No duplicate dates
Iteration for store CA_1 and category HOUSEHOLD:
No duplicate dates
Iteration for store CA_2 and category FOODS:
No duplicate dates
Iteration for store CA_2 and category HOBBIES:
No duplicate dates
Iteration for store CA_2 and category HOUSEHOLD:
No duplicate dates
Iteration 

# Answers to question 5

In [64]:
complete_date_range=pd.date_range(start=min_date,end=max_date)
for i,j in product(stores,categories):
    print(f"Iteration for store {i} and category {j}:")
    control_df=df[(df['store_id']==i) & (df['cat_id']==j)]
    missing_dates=complete_date_range[~complete_date_range.isin(control_df['date'])]
    if len(missing_dates)==0:
        print('There are no missing dates')
    else:
        missing_dates

Iteration for store TX_1 and category FOODS:
There are no missing dates
Iteration for store TX_1 and category HOBBIES:
There are no missing dates
Iteration for store TX_1 and category HOUSEHOLD:
There are no missing dates
Iteration for store TX_2 and category FOODS:
There are no missing dates
Iteration for store TX_2 and category HOBBIES:
There are no missing dates
Iteration for store TX_2 and category HOUSEHOLD:
There are no missing dates
Iteration for store TX_3 and category FOODS:
There are no missing dates
Iteration for store TX_3 and category HOBBIES:
There are no missing dates
Iteration for store TX_3 and category HOUSEHOLD:
There are no missing dates
Iteration for store CA_1 and category FOODS:
There are no missing dates
Iteration for store CA_1 and category HOBBIES:
There are no missing dates
Iteration for store CA_1 and category HOUSEHOLD:
There are no missing dates
Iteration for store CA_2 and category FOODS:
There are no missing dates
Iteration for store CA_2 and category HO

# Answers to question 6

In [65]:
# detrend each time series
df['detrended_sales']=np.nan
for i,j in product(stores,categories):
    #print(f"Iteration for store {i} and category {j}:")
    control_df=df[(df['store_id']==i) & (df['cat_id']==j)]
    df.loc[(df['store_id']==i) & (df['cat_id']==j),'detrended_sales']=detrend(control_df['sales'])

In [66]:
# outlier detection function
def outlier_detection(df):
    """
    Add column 'anomaly' to dataframe to mark outliers as True, non-outliers as False. 
    """
    df['anomaly']=False
    for i,j in product(stores,categories):
        st_cat_mask=(df['store_id']==i) & (df['cat_id']==j)
        subset = df[st_cat_mask].copy()
        q1=df.loc[st_cat_mask,'detrended_sales'].quantile(q=0.25)
        q3 = df.loc[st_cat_mask,'detrended_sales'].quantile(q= 0.75)
        iqr = q3-q1
        higher_bound = q3 + 1.5*iqr
        lower_bound = q1 - 1.5*iqr
        outlier_mask = (df.loc[st_cat_mask,'detrended_sales']>higher_bound) | (df.loc[st_cat_mask,'detrended_sales']<lower_bound)
        df.loc[st_cat_mask, 'anomaly'] = outlier_mask.values
    return df

In [67]:
# execute outlier detection function for each time series
df=outlier_detection(df)

In [68]:
df[df['anomaly']==True]

,date,store_id,cat_id,sales,detrended_sales,anomaly
330,2011-12-25,TX_1,FOODS,0.00,-4325.732392,True
434,2012-04-07,TX_1,FOODS,8212.22,3818.767400,True
435,2012-04-08,TX_1,FOODS,8057.01,3662.906244,True
442,2012-04-15,TX_1,FOODS,7137.98,2739.318154,True
463,2012-05-06,TX_1,FOODS,7965.78,3553.443881,True
...,...,...,...,...,...,...
58137,2016-02-20,WI_3,HOUSEHOLD,4428.43,1835.468312,True
58145,2016-02-28,WI_3,HOUSEHOLD,4226.09,1631.292909,True
58152,2016-03-06,WI_3,HOUSEHOLD,4220.48,1624.076931,True
58178,2016-04-01,WI_3,HOUSEHOLD,4256.42,1654.051871,True


In [69]:
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", font_size=15, title_font_size=20):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=True,
            width=900,
            height=500,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 15
            },
            legend_title = None,
            legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            )
        )
    return fig

In [70]:
# sample one of the store category to see how our outlier detection performs
def visualize_outliers(store, category):
    # visualization of outliers detected
    font_size=10
    store_category_mask=(df['store_id']==store) & (df['cat_id']==category)
    outlier_mask=(df['store_id']==store) & (df['cat_id']==category) & (df['anomaly']==True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df.loc[store_category_mask,'date'],
        y=df.loc[store_category_mask,'detrended_sales'],
        mode="lines",
        name="Detrended sales"
    ))
    fig.add_trace(go.Scatter(
        x=df.loc[outlier_mask,'date'],
        y=df.loc[outlier_mask,'detrended_sales'],
        mode='markers',
        marker_symbol="star",
        marker_size=5,
        name="Outliers"
    ))
    fig.update_layout(
        title_text=f"Store {store} and category {category}:# of Outliers: {outlier_mask.sum()} | % of Outliers: {outlier_mask.sum()/len(df.loc[store_category_mask,'detrended_sales'])*100:.2f}%",
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        )
    )
    return fig



In [71]:
# sample one of the store category to see how our outlier detection performs
fig = visualize_outliers("TX_1","FOODS")
format_plot(fig, xlabel="Time", ylabel="Detrended sales")
fig.show()

It turns out that values shown on 25 december of every year are anomalies which deserve to be assigned zeros as those dates refer to Christmas

The following function will take charge of this task.

In [72]:
def christmas_impute(df):
    christmas_mask=(df['date']==2011-12-25) & (df['date']==2012-12-25) & (df['date']==2013-12-25) & (df['date']==2014-12-25) & (df['date']==2015-12-25)
    df.loc[christmas_mask,'sales']=0
    return df

In [73]:
df=christmas_impute(df)

In [74]:
#set our outliers to null sales value and count them as missing values in the following question
outlier_global_mask=(df['anomaly']==True)
df.loc[outlier_global_mask,'sales']=np.nan

# Answers to question 7

In [75]:
# count the global number of missing values
total_number_missing=df.loc[:,'sales'].isna().sum()

In [79]:
print(f'The total number of missing values in time series are: {total_number_missing} points')

The total number of missing values in time series are: 1143 points


In [80]:
# Now let's analyse the ratio of missing values in each time series
combination=[]
number_missing_values=[]
ratio_missing_values=[]
for i,j in product(stores,categories):
    combination.append(f"Store: {i}-- Category: {j}")
    mask=(df['store_id']==i) & (df['cat_id']==j)
    number_missing_values.append(df.loc[mask,'sales'].isna().sum())
    ratio_missing_values.append(df.loc[mask,'sales'].isna().sum()/len(df.loc[mask,'sales']))
missing_stats_df=pd.DataFrame({"Time series":combination,
                               "# missing values":number_missing_values,
                              "% missing values":ratio_missing_values})

In [81]:
missing_stats_df

,Time series,# missing values,% missing values
0,Store: TX_1-- Category: FOODS,22,0.011334
1,Store: TX_1-- Category: HOBBIES,39,0.020093
2,Store: TX_1-- Category: HOUSEHOLD,35,0.018032
3,Store: TX_2-- Category: FOODS,21,0.010819
4,Store: TX_2-- Category: HOBBIES,30,0.015456
5,Store: TX_2-- Category: HOUSEHOLD,29,0.014941
6,Store: TX_3-- Category: FOODS,14,0.007213
7,Store: TX_3-- Category: HOBBIES,51,0.026275
8,Store: TX_3-- Category: HOUSEHOLD,32,0.016486
9,Store: CA_1-- Category: FOODS,11,0.005667


# Answers to question 8

In [84]:
# Reminder: Outliers have already been set to null values in question 6
# Now let's impute each time series missing values using linear interpolation as required
def missing_imputation(df):
    """
    Impute missing values in each time series conained in df dataframe. 
    """
    for i,j in product(stores,categories):
        st_cat_mask=(df['store_id']==i) & (df['cat_id']==j)
        subset = df[st_cat_mask].copy()
        subset['sales']=subset['sales'].interpolate()
        #missing_mask =(df['store_id']==i) & (df['cat_id']==j) & (df['sales'].isna())
        df.loc[st_cat_mask, 'sales'] = subset['sales'].values
    return df

In [85]:
df=missing_imputation(df)

In [87]:
#Recheck wether there are missing values 
df.loc[:,'sales'].isna().sum()

0

# Answers to question 9

In [88]:
#Check the form of the dataframe before saving
df.head()

,date,store_id,cat_id,sales,detrended_sales,anomaly
0,2011-01-29,TX_1,FOODS,3950.35,-160.500963,False
1,2011-01-30,TX_1,FOODS,3844.97,-266.532119,False
2,2011-01-31,TX_1,FOODS,2888.03,-1224.123274,False
3,2011-02-01,TX_1,FOODS,3631.28,-481.524430,False
4,2011-02-02,TX_1,FOODS,3072.18,-1041.275586,False


In [91]:
# Drop 'detrended sales' and 'anomaly' columns as they will not be needed in the future
df=df.drop(['detrended_sales','anomaly'],axis=1)

In [93]:
df.to_csv(str(data_path)+'/'+r'/sales_processed.csv', index = False)